In [27]:
import sys
sys.path.append('./privatizers')
sys.path.append('./dataProviders')
sys.path.append('./adapters')

from sklearn import tree
from sklearn.datasets import fetch_covtype
from adapters import generalAdapter as adapter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import numpy as np

dp = fetch_covtype()
X = dp.data
Y = dp.target

In [28]:
# ## Size adjustment
# desired_size = 10000
# min_test_size = 0.3
# test_size = 0.3

# total_size = len(X)

# if (desired_size < min_test_size * total_size):
#     test_size = 1-(desired_size / total_size);

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

# ## Dataset
# data_set = {
#     'samples': X_train,
#     'labels': y_train
# }

In [29]:
## Multiple Dataset sizes
desired_sizes = [100, 1000, 10000, 100000]
data_sets = []

min_test_size = 0.2

total_size = len(X)

for desired_size in desired_sizes:
    test_size = 0.2
    if (desired_size < (1-min_test_size) * total_size):
        test_size = 1-(desired_size / total_size);

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

    ## Dataset
    single_data_set = {
        'samples': X_train,
        'labels': y_train
    }
    data_sets.append(single_data_set)

In [30]:
for data_set in data_sets:
    ## Decision tree without privatization
    clf = tree.DecisionTreeClassifier()
    clf.fit(data_set['samples'], data_set['labels'])

    y_pred = clf.predict(X_test)

    print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

    ## With cross validation
    # scores = cross_val_score(clf, X, Y, cv=5)
    # for v in scores:
    #     print(str(v).replace('.', ','))

0,5729387771224478
0,6199686910748774
0,7308970859415442
0,8750407993131163


In [31]:
from privatizers import laplacePrivatizer
for data_set in data_sets:
    ## Privatization: Laplace privatizer

    ad = adapter.GeneralAdapter(7, 1)

    dataInput = ad.fromRaw(data_set['samples'])
    dataTarget = ad.fromRaw(data_set['labels'])

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)

    inputSensitivity = privatizer.getSensitivityList(dataInput)
    targetSensitivity = 7

    epsilon = [1.0, 1.5, 2.0, 2.5, 3.0]

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)
    privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)
    for i in epsilon:
        privatizer = laplacePrivatizer.LaplacePrivatizer(i)
        privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)

        privateTargetsFloat = privatizer.privatize(dataTarget, sensitivityList = targetSensitivity)
        privateTargets = ad.toDiscreteValue(privateTargetsFloat)

        clf = tree.DecisionTreeClassifier()
        clf.fit(privateData, privateTargets)
        y_pred = clf.predict(X_test)

        print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

0,4790993174820639
0,46842600927625655
0,3741146289185531
0,39986237378199757
0,5054915355718037
0,48798889011315705
0,3670171076457393
0,5103832952539744
0,6364734425057119
0,6025304929388603
0,4560812285738639
0,4850492606228938
0,5415570888936474
0,5318338589601529
0,5929756576225591
0,49882227715259253
0,4893214944294645
0,5800404562870443
0,5697143320450798
0,5479872685353618
